# Example 2: Running PhysiCell simulations with summary functions

This notebook demonstrates how to execute PhysiCell simulations using the UQ_PhysiCell framework with different data processing approaches. The example uses the `physicell_model_2` configuration (which uses the [virus-macrophage](https://github.com/MathCancer/PhysiCell/tree/master/sample_projects/virus_macrophage) PhysiCell model) from [Model_Struct.ini](Model_Struct.ini) and explores three different simulation execution modes to showcase the flexibility of output processing.

The notebook covers three simulation approaches:
1. **Raw simulation output**: Running simulations without summary functions to generate complete MCDS (MultiCellular Data Standard) time series data
2. **Predefined summary function**: Using built-in summary functions like `summ_func_FinalPopLiveDead` for standardized data extraction
3. **Custom summary function**: Implementing user-defined functions to extract specific quantities of interest (QoIs) and perform custom data processing

Each approach demonstrates different use cases: raw output for detailed analysis, predefined functions for common metrics, and custom functions for specialized research questions. The parameters explored include:
- **viral_replication_rate**: Rate of virus replication inside epithelial cells
- **min_virion_count**: Minimum number of virions required for macrophage recognition

In [1]:
from uq_physicell import PhysiCell_Model
from uq_physicell.utils import summ_func_FinalPopLiveDead
import numpy as np

fileName = "Model_Struct.ini"
key_model = "physicell_model_2"
# Create the structure of model exploration
PhysiCellModel = PhysiCell_Model(fileName, key_model, verbose=True)

	> Constructor PhysiCell_Model: physicell_model_2 at Model_Struct.ini...
		>> Reading config file: Model_Struct.ini ...
		>> Checking executable format ...
		>> Checking parameters in XML file ...
		>> Checking parameters in RULES file ...


## First simulation with no summary function

In [2]:
# It will generate a folder output inside of output_2
PhysiCellModel.RunModel(SampleID=0, ReplicateID=0,Parameters={"viral_replication_rate": 0.75, "min_virion_count": 0.5})

	> Running - Sample:0, Replicate: 0, Parameters XML: {'viral_replication_rate': 0.75, 'min_virion_count': 0.5}, Parameters rules: {}...
		>> Setting up model input ...
			>>> Checking parameters input ...
				>>>> Checking XML parameters input <class 'dict'> ...
				>>>> Checking RULES parameters type <class 'dict'> ...
			>>> Setting up XML input...
			>>> Generating XML file config2/config_S000000_00.xml ...
		>> Running model ...
Registered process 0_0 with PID 77361
		>> Removing config files ...


## Second simulation using the summary function summ_func_FinalPopLiveDead.

In [3]:
PhysiCellModel.RunModel(SampleID=0, ReplicateID=0,Parameters={"viral_replication_rate": 0.75, "min_virion_count": 0.5},SummaryFunction=summ_func_FinalPopLiveDead)

	> Running - Sample:0, Replicate: 0, Parameters XML: {'viral_replication_rate': 0.75, 'min_virion_count': 0.5}, Parameters rules: {}...
		>> Setting up model input ...
			>>> Checking parameters input ...
				>>>> Checking XML parameters input <class 'dict'> ...
				>>>> Checking RULES parameters type <class 'dict'> ...
			>>> Setting up XML input...
			>>> Generating XML file config2/config_S000000_00.xml ...
		>> Running model ...
Registered process 0_0 with PID 77408
		>> Removing config files ...
		>> Running summary function: <function summ_func_FinalPopLiveDead at 0x11e40b100> ...
			>>> Returning summary data ...


,time,sampleID,replicateID,live_cells,dead_cells,run_time_sec,viral_replication_rate,min_virion_count
0,2880.01,0,0,50,0,20.119688,0.75,0.5


## Third simulation using a custom summary function 

In [4]:
import pcdl
import pandas as pd
from shutil import rmtree

def custom_summary_func(OutputFolder:str,SummaryFile:str, dic_params:dict, SampleID:int, ReplicateID:int):
    mcds_ts = pcdl.TimeSeries(OutputFolder, microenv=False, graph=False, settingxml=None, verbose=False)
    for mcds in mcds_ts.get_mcds_list():
        df_cell = mcds.get_cell_df()
        live_cells = len(df_cell[ (df_cell['dead'] == False) ] )
        dead_cells = len(df_cell[ (df_cell['dead'] == True) ] )
        data = {'time': mcds.get_time(), 'sampleID': SampleID, 'replicateID': ReplicateID, 'live_cells': live_cells, 'dead_cells': dead_cells, 'run_time_sec': mcds.get_runtime()}
        data_conc = {**data,**dic_params} # concatenate output data and parameters
        if ( mcds.get_time() == 0 ): df = pd.DataFrame([data_conc]) # create the dataframe
        else: df.loc[len(df)] = data_conc # append the dictionary to the dataframe
   # remove replicate output folder
    rmtree( OutputFolder )
    if (SummaryFile):
        df.to_csv(SummaryFile, sep='\t', encoding='utf-8')
        return None
    else: return df

PhysiCellModel.RunModel(SampleID=1, ReplicateID=0,Parameters={"viral_replication_rate": 0.80, "min_virion_count": 0.55}, SummaryFunction=custom_summary_func)

	> Running - Sample:1, Replicate: 0, Parameters XML: {'viral_replication_rate': 0.8, 'min_virion_count': 0.55}, Parameters rules: {}...
		>> Setting up model input ...
			>>> Checking parameters input ...
				>>>> Checking XML parameters input <class 'dict'> ...
				>>>> Checking RULES parameters type <class 'dict'> ...
			>>> Setting up XML input...
			>>> Generating XML file config2/config_S000001_00.xml ...
		>> Running model ...
Registered process 1_0 with PID 78228
		>> Removing config files ...
		>> Running summary function: <function custom_summary_func at 0x14ef62700> ...
			>>> Returning summary data ...


,time,sampleID,replicateID,live_cells,dead_cells,run_time_sec,viral_replication_rate,min_virion_count
0,0.0,1,0,1060,0,0.000008,0.8,0.55
1,360.0,1,0,50,0,5.718475,0.8,0.55
2,720.0,1,0,50,0,7.785989,0.8,0.55
3,1080.0,1,0,50,0,9.892772,0.8,0.55
4,1440.0,1,0,50,0,12.036282,0.8,0.55
5,1800.0,1,0,50,0,14.214356,0.8,0.55
6,2160.0,1,0,50,0,16.376806,0.8,0.55
7,2520.0,1,0,50,0,18.623699,0.8,0.55
8,2880.0,1,0,50,0,20.752024,0.8,0.55
